In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plot
import math

## 기본 환경 구성

In [2]:
MC_N = 10**4   # 데이터 생성 시뮬레이션 횟수
MC_N = MC_N

size_x = 25   # x축 사이즈
size_y = 25   # y축 사이즈

X_x_list = np.random.rand(MC_N,1) * size_x    # 0과 25 사이 임의의 타겟 데이터의 x값 생성
X_x_list = X_x_list.flatten().tolist()
X_y_list = np.random.rand(MC_N,1) * size_y    # 0과 25 사이 임의의 타겟 데이터의 y값 생성
X_y_list = X_y_list.flatten().tolist()

sensor_ref_x_list = [0] * MC_N                      #sensor ref x coordinate
sensor_ref_y_list = [0] * MC_N                      #sensor ref  y coordinate
sensor_one_x_list = [size_x] * MC_N                 #sensor 1 x coordinate
sensor_one_y_list = [0] * MC_N                      #sensor 1 y coordinate
sensor_two_x_list = [0] * MC_N                      #sensor 2 x coordinate
sensor_two_y_list = [size_y] * MC_N                 #sensor 2 y coordinate
sensor_three_x_list = [size_x] * MC_N                #sensor 3 x coordinate
sensor_three_y_list = [size_y] * MC_N                #sensor 3 y coordinate


environment_error_rate = 0.5

## 실제 타겟과 기기가 관측한 타겟 데이터 생성

In [3]:
max_device_error = 0.1  #위의 사진에 의해 device 기기의 에러값 선언 (미터)

noise_x_list = []
noise_y_list = []


while(len(noise_x_list) != MC_N):
    #평균0 표준편차가 1인 가우시안 표준정규분포 난수 생성 뒤 max_device_error을 곱하여 맥시멈 에러에 대해 스케일링
    temp_x = np.random.randn()  * max_device_error   
    temp_y = np.random.randn()  * max_device_error
    
    if( (temp_x**2 + temp_y**2 < max_device_error ** 2) ):    
        noise_x_list.append(temp_x)
        noise_y_list.append(temp_y)
    
    
X_hat_x_list = [x+y for x,y in zip(X_x_list, noise_x_list)]   #기기를 통해 생성되는 측정되는 좌표의 x값 
X_hat_y_list = [x+y for x,y in zip(X_y_list, noise_y_list)]   #기기를 통해 생성되는 측정되는 좌표의 y값 

## TOA값 생성

In [4]:
distance_ref_X_list = [-100] * MC_N  
distance_one_X_list = [-100] * MC_N  
distance_two_X_list = [-100] * MC_N 
distance_three_X_list = [-100] * MC_N  

distance_ref_X_hat_list = [-100] * MC_N  
distance_one_X_hat_list = [-100] * MC_N  
distance_two_X_hat_list = [-100] * MC_N  
distance_three_X_hat_list = [-100] * MC_N  


for i in range(MC_N):
    X_x = X_x_list[i]
    X_y = X_y_list[i]
    
    X_hat_x = X_hat_x_list[i]
    X_hat_y = X_hat_y_list[i]
    
    sensor_ref_x = sensor_ref_x_list[i]
    sensor_ref_y = sensor_ref_y_list[i]
    sensor_one_x = sensor_one_x_list[i]
    sensor_one_y = sensor_one_y_list[i]
    sensor_two_x = sensor_two_x_list[i]
    sensor_two_y = sensor_two_y_list[i]
    sensor_three_x = sensor_three_x_list[i]
    sensor_three_y = sensor_three_y_list[i]
    
    distance_ref_X_hat_list[i] = math.sqrt((X_hat_x - sensor_ref_x)**2 + (X_hat_y - sensor_ref_y)**2)  # ref과 Xhat 거리
    distance_one_X_hat_list[i] = math.sqrt((X_hat_x - sensor_one_x)**2 + (X_hat_y - sensor_one_y)**2)  # 1과 Xhat 거리
    distance_two_X_hat_list[i] = math.sqrt((X_hat_x - sensor_two_x)**2 + (X_hat_y - sensor_two_y)**2)  # 2와 Xhat 거리
    distance_three_X_hat_list[i] = math.sqrt((X_hat_x - sensor_three_x)**2 + (X_hat_y - sensor_three_y)**2)  # 3과 타Xhat 거리

In [5]:
df = pd.DataFrame(data = {'X_x':X_x_list , 'X_y' : X_y_list , 
                         'X_noise_x':noise_x_list , 'X_noise_y' : noise_y_list , 
                          'X_hat_x':X_hat_x_list ,'X_hat_y':X_hat_y_list,
                          'distance_ref_X_hat' : distance_ref_X_hat_list , 'distance_one_X_hat' : distance_one_X_hat_list,
                          'distance_two_X_hat' : distance_two_X_hat_list , 'distance_three_X_hat' : distance_three_X_hat_list   
                         })

## 환경에 의한 에러값 생성

http://preview.kyobobook.co.kr/epubPreviewPopup.jsp?type=web&barcode=480D130703480&search=Y
<br>
https://hyoukjang.github.io/vr/indoor-positioning/ips/2017/10/22/positional-tracking-101-2.html
<br>
마이크로파 파장 : 약 10^-2

In [6]:
def create_environment_error(distance):
    error_rate = 0.3
    if(float(np.random.rand()) < error_rate):
        obstacle_heigth =  np.random.rand()   #부딪히는 물체의 길이
        if(distance < obstacle_heigth):         # 말이 안되는거 그냥 0으로 채워 넣음
            return 0
        
        #By 프레넬 영역을 이용한 추가 이동 거리 구현(함 위에 사이트 읽어보고 이상하다 싶으면 바로 말해줘요)
        distance1 = np.random.randn() + (distance/2)
        distance2 = distance - distance1
        if(distance1 < 0 or distance2 < 0):     # 말이 안되는거 그냥 0으로 채워 넣음
            return 0 
        
        Fresnel_a = 0.01 * math.sqrt(distance1 * distance2 / (distance1+distance2))

        break_n = 1
        while(Fresnel_a < obstacle_heigth):
            Fresnel_a = Fresnel_a * (break_n+1) / break_n
            break_n +=1
        
        distance_final = math.sqrt(np.exp2(distance1) + np.exp2(Fresnel_a)) + math.sqrt(np.exp2(distance2) + np.exp2(Fresnel_a))
        
        # 너무 이상한값 우리선에서 정리
        if(distance_final > 100):
            distance_final = 100
        
    else:
        distance_final = distance

    return distance_final

In [7]:
df['error_distance_ref_X_hat'] = df['distance_ref_X_hat'].apply(create_environment_error)
df['error_distance_one_X_hat'] = df['distance_one_X_hat'].apply(create_environment_error)
df['error_distance_two_X_hat'] = df['distance_two_X_hat'].apply(create_environment_error)
df['error_distance_three_X_hat'] = df['distance_three_X_hat'].apply(create_environment_error)


In [8]:
df['distance_ref_X_hat'].describe()

count    10000.000000
mean        19.176010
std          7.074350
min          0.261905
25%         14.224513
50%         20.027954
75%         24.392928
max         35.224946
Name: distance_ref_X_hat, dtype: float64

In [9]:
df['error_distance_ref_X_hat'].describe()

count    10000.000000
mean        32.000908
std         28.769702
min          0.000000
25%         15.333763
50%         22.195751
75%         29.618585
max        100.000000
Name: error_distance_ref_X_hat, dtype: float64

## TDOA값 생성

In [11]:
df['TDOA_X_hat_ref_one_error'] = df['error_distance_ref_X_hat'] - df['error_distance_one_X_hat']
df['TDOA_X_hat_ref_two_error'] = df['error_distance_ref_X_hat'] - df['error_distance_two_X_hat']
df['TDOA_X_hat_ref_three_error'] = df['error_distance_ref_X_hat'] - df['error_distance_three_X_hat']

In [12]:
df_parsing = df[['TDOA_X_hat_ref_one_error' , 'TDOA_X_hat_ref_two_error' , 'TDOA_X_hat_ref_three_error' , 'X_hat_x' , 'X_hat_y']]
df_parsing.columns = ['TDOA_1', 'TDOA_2', 'TDOA_3' , 'coordinate_x' , 'coordinate_y']

df_parsing = df_parsing.reset_index(drop=True)

In [13]:
df_parsing

,TDOA_1,TDOA_2,TDOA_3,coordinate_x,coordinate_y
0,78.949539,79.238811,93.480068,20.270349,20.512247
1,3.777966,9.539378,14.793111,16.186692,20.709705
2,-77.443970,20.100752,-2.340767,0.223558,22.554922
3,45.979282,-6.993135,44.907504,15.477554,11.826099
4,-0.454112,-54.492988,-5.831061,12.224295,8.608425
...,...,...,...,...,...
9995,-2.131181,-9.256385,-10.558660,11.315802,6.037584
9996,12.965217,2.536776,13.586878,23.789383,15.237974
9997,13.995493,-6.535459,2.098095,20.714582,6.007455
9998,-80.963625,12.556311,-4.555417,2.208372,18.907847
